<a href="https://colab.research.google.com/github/paucaroscanoa/ApiBookAuthor/blob/master/Caso_de_Estudio_2_3_%C2%BFCrecen_m%C3%A1s_r%C3%A1pido_los_pa%C3%ADses_pobres_que_los_ricos%3F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de Estudio 2.3 - ¿Crecen más rápido los países pobres que los ricos?


Sincronice su cuenta de Google. Para ello, siga el link que aparece en la salida de la siguiente celda una vez ejecutada. Copie el código que le aparece en pantalla e introdúzcalo en la salida de la celda. Una vez vea el mensaje: `Google Drive sincronizado con éxito!` puede continuar ejecutando el resto de celdas.

In [ ]:
from google.colab import auth
auth.authenticate_user()

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
data_drop = drive.CreateFile({'id':'1wAZ5JZuq2_zKqzDQlIuOxUSTfk9N_tq5'})
data_drop.GetContentFile('growth.Rdata')

print('Google Drive sincronizado con éxito!')

In [ ]:
import rpy2
import rpy2.robjects

In [ ]:
%load_ext rpy2.ipython

Instalar e importar librerías: (ignorar resultados a no ser que no se muestre la frase: `¡Librerías instaladas con éxito!`)

In [ ]:
%%R
install.packages("hdm")

cat('\n¡Librerías instaladas con éxito!')

In [ ]:
%%R
# Importar librerías
library(hdm)

cat('\n¡Librerías importadas con éxito!')

## Datos


In [ ]:
%%R
# Cargar la base de datos
load(file='growth.Rdata')

# Ver las variables de la base de datos
class(growth)
str(growth)

# Mostrar dimensiones de la base de datos
dims  <- dim(growth)
cat('\nDimensiones de la base de datos:',toString(dims),'\n',fill = TRUE)

In [ ]:
%%R
write.csv(growth,'prueba.csv')


In [ ]:
%%R
# Obtener los nombres de las variables
varnames= colnames(growth)
# Extraer los nombres de los controles y del regresor objetivo de varnames
xnames     <- varnames[-c(1,2,3)]     # nombres de las variables de X
dandxnames <- varnames[-c(1,2)]       # nombres de las variables de X y D

## Metodología

In [ ]:
%%R
# crear fórmulas "pegando" nombres (así se guardan los nombre a mostrar)
fmla      <-  as.formula(paste("Outcome ~ ", paste(dandxnames, collapse= "+")))
full.fit  <-  lm(fmla, data=growth)
fmla.y    <-  as.formula(paste("Outcome ~ ", paste(xnames, collapse= "+")))
fmla.d    <-  as.formula(paste("gdpsh465~ ", paste(xnames, collapse= "+")))


# d parcial e y parcial por regresión lineal
rY       <- rlasso(fmla.y, data =growth)$res
rD       <- rlasso(fmla.d, data =growth)$res


# regresión lineal de Y parcialmente extraida en la D parcialmente extraída
partial.fit.lasso <- lm(rY~rD-1)

# crear tabla para recoger los resultados
table      <- matrix(0, 2, 4)
table[1,1:2]  <- summary(full.fit)$coef["gdpsh465",1:2]
table[2,1:2]  <- summary(partial.fit.lasso)$coef[1,1:2]
table[1,3:4]  <- confint(full.fit, level=0.95, 'gdpsh465')[1:2]
table[2,3:4]  <- confint(partial.fit.lasso, level=0.95, 'rD')[1:2]

# asignar nombre de columnas y filas
colnames(table) <- c("Estimación (beta)", "Error estándar","2.5%","97.5%")
rownames(table) <- c("Mínimos cuadrados ord. (OLS)", "Extracción parcial con Lasso")

## Resultados

In [ ]:
%%R
# Mostrar resultados
print(table, digits=2)